# KING

In [57]:
from pandas import concat, read_csv, to_datetime
teams = read_csv('./data/2015/team.csv', index_col=0)
statistics = read_csv('./data/2015/team-game-statistics.csv', index_col=0)
data = teams.join(statistics)
for column in data:
    if 'Unnamed' in column:
        del data[column]
data.Date = to_datetime(data.Date, format='%Y%m%d')
data

,Name,Conference Code,Game Code,Rush Att,Rush Yard,Rush TD,Pass Att,Pass Comp,Pass Yard,Pass TD,...,Penalty Yard,Third Down Att,Third Down Conv,Fourth Down Att,Fourth Down Conv,Red Zone Att,Red Zone TD,Red Zone Field Goal,First Down Total,Date
Team Code,,,,,,,,,,,,,,,,,,,,,
2,Abilene Christian,99005,2.009620e+12,29,34,1,45,30,213,1,...,40,17,5,4,2,0,0,0,15,2015-09-04
5,Akron,875,5.052220e+12,44,138,0,26,6,88,0,...,62,19,4,0,0,0,0,0,10,2015-09-05
5,Akron,875,5.450005e+14,24,42,1,22,7,63,0,...,85,11,2,0,0,0,0,0,8,2015-09-12
5,Akron,875,6.320005e+14,47,291,5,33,18,335,2,...,51,13,7,1,0,0,0,0,31,2015-09-19
5,Akron,875,5.067120e+12,42,220,3,29,13,238,2,...,92,18,9,1,0,0,0,0,22,2015-09-26
5,Akron,875,5.190005e+14,35,88,0,38,18,231,0,...,79,17,5,3,2,0,0,0,21,2015-10-03
5,Akron,875,5.020420e+12,53,278,2,22,14,222,4,...,54,16,11,0,0,0,0,0,28,2015-10-10
5,Akron,875,5.007120e+12,41,158,0,30,15,104,1,...,70,18,6,3,1,0,0,0,16,2015-10-17
5,Akron,875,1.290005e+14,25,25,0,36,22,252,1,...,34,16,5,0,0,0,0,0,15,2015-10-31
